In [1]:
from pointCluster import PointCluster
from csvAnalysis import CSVAnalysis
from placesGeometry import PlacesGeometry
from poi import Poi
from toGeoPackage import ToGeoPackage

print("Optimal Placement of EV Charging Stations in Georgia State")

# Declare data file paths
poiCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\georgia_pois.csv')
vehicleCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\vehicle_hotspot_pois.csv')
placesCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\places_coords.csv')
county_info = placesCSV.readPlaces('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\county_pop_dens.csv', 'C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\county_inc.csv')

# Read data files and extract POIs
poi_lat_lons = poiCSV.createLatLons()
raw_lat_lons = vehicleCSV.rawPoints()
vehicle_lat_lons = vehicleCSV.createLatLons()
vehicle_pois = vehicleCSV.readPoints()

# First run of clustering based on popular locations
raw_counties = PointCluster.cluster_counties(county_info, poi_lat_lons)

# Determine which counties do not have EV points within them
remaining_counties = PointCluster.cluster_counties(PlacesGeometry.find_remaining_counties(poi_lat_lons, county_info), poi_lat_lons)

# Filter parking locations to have fewer points
vehicle_lat_lons = PointCluster.filter_close_points(vehicle_lat_lons)

# Cluster the combination of popular POIs and vehicle POIs
cluster = PointCluster()
all_coords = PointCluster.combine_clusters(poi_lat_lons, vehicle_lat_lons)
cluster.coords = all_coords
cluster.point_cluster()

# Adjust pois to vehicle hotspots
adjusted_popular = cluster.adjust_points(vehicle_lat_lons, vehicle_pois)

# Determine hotspot capacities based on filtered points
vehicle_pois = PointCluster.assign_weights(adjusted_popular, vehicle_pois)

# Add chargers to promote diversity and accessibility
diversity = PointCluster.assign_weights(PointCluster.adjust_chargers(raw_counties, vehicle_lat_lons), vehicle_pois)
remaining = PointCluster.assign_weights(PointCluster.adjust_chargers(remaining_counties, vehicle_lat_lons), vehicle_pois)

finals = vehicle_pois + diversity + remaining
finals = Poi.correct_names(finals)

# Create new ToGeoPackage object to write file
toGeoPackage = ToGeoPackage()
toGeoPackage.extract_data(finals)
toGeoPackage.write_file()

Optimal Placement of EV Charging Stations in Georgia State
Algorithm v1.1.0


Reading csv file...
|████████████████████████████████████████| 31416/31416 [100%] in 0.3s (110484.96
Extracting latitude longitude...
|████████████████████████████████████████| 31416/31416 [100%] in 0.1s (356034.66
Reading csv file...
|████████████████████████████████████████| 331716/331716 [100%] in 2.5s (136727.
Reading csv file...
|████████████████████████████████████████| 331716/331716 [100%] in 2.2s (151685.
Extracting latitude longitude...
|████████████████████████████████████████| 331716/331716 [100%] in 0.8s (425433.
Reading csv file...
|████████████████████████████████████████| 331716/331716 [100%] in 2.5s (133261.
Reading coordinate file...
|████████████████████████████████████████| 1059/1059 [100%] in 0.5s (2829.38/s) 
Reading population file...
Reading income file...
Identifying diversity counties...
Calculating additional chargers...
|████████████████████████████████████████| 44/44 [100%] in 6.5s